In [1]:
# Imports
from datetime import datetime
import pandas as pd
import numpy as np
import tweepy as tw
import preprocessor as p
from os import environ as env

In [2]:
# Read in senator usernames and parties
usernames_df = pd.read_csv("senators_usernames.csv")
usernames_list = usernames_df.username.tolist()

In [3]:
# Load credentials from .env file and authorize
auth = tw.AppAuthHandler(env["API_KEY"], env["API_KEY_SECRET"])
api = tw.API(auth)

In [4]:
# Initialize empty lists
dfs_list = []
failed_list = []

# Get raw tweets
for name in usernames_list:
    try:
        # Send a query
        tweets = api.user_timeline(screen_name=name, tweet_mode="extended", count=1000)

        # Post-process tweets
        tweets_json = [t._json for t in tweets]
        tweets_normalized = pd.json_normalize(tweets_json)
        dfs_list.append(tweets_normalized)
    except Exception:
        print(f"Unable to find tweets for {name}")
        failed_list.append(name)


In [5]:
# Combine all dfs into one df and clean
tweets_raw_df = pd.concat(dfs_list)

In [6]:
# Get size of dataframe
print(tweets_raw_df.shape)

# Peak at raw tweets
tweets_raw_df.head(3)

(19999, 360)


,created_at,id,id_str,full_text,truncated,display_text_range,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,retweeted_status.quoted_status.quoted_status_permalink.expanded,retweeted_status.quoted_status.quoted_status_permalink.display,retweeted_status.geo.type,retweeted_status.geo.coordinates,retweeted_status.coordinates.type,retweeted_status.coordinates.coordinates,quoted_status.geo.type,quoted_status.geo.coordinates,quoted_status.coordinates.type,quoted_status.coordinates.coordinates
0,Wed Sep 14 18:17:07 +0000 2022,1570114434529644545,1570114434529644545,The Inflation Reduction Act will be life-chang...,False,"[0, 202]","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wed Sep 14 16:17:52 +0000 2022,1570084427148951553,1570084427148951553,Thanks to @Right4LGBTQ for the support we are ...,False,"[0, 144]","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tue Sep 13 23:59:35 +0000 2022,1569838232623366144,1569838232623366144,Thanks to @POTUS for having me @WhiteHouse tod...,False,"[0, 279]","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Let's clean some tweets! 
tweets_df = (
    tweets_raw_df
    .filter(["created_at", "id", "full_text", "user.screen_name"])                                   # Keep only relevant columns
    .assign(id=tweets_raw_df["id"].astype(str),                                                      # Convert id to string
            created_at=tweets_raw_df["created_at"].apply(pd.to_datetime),                            # Convert datetime to timestamp with Pandas method
            text=tweets_raw_df["full_text"].apply(p.clean).str.replace("&amp;", "and "),             # Clean tweets using preprocessor and replace &amp; with "and "
            text_length=tweets_raw_df["full_text"].apply(len))                                       # Calculate length of tweets
    .merge(usernames_df[["username", "party"]], left_on="user.screen_name", right_on="username")     # Merge with usernames_df to get politcal affiliations                                                # Get usernames, party affiliations
    .drop(columns=["username"])                                                                      # Drop extra username column added after merge
    .drop_duplicates(["text"])                                                                       # Drop duplicate tweets
    .rename({"user.screen_name": "username"}, axis=1)                                                # Rename columns
    .sort_values(by="created_at", ascending=False)                                                   # Sort tweets by date
    .reindex(columns=["created_at", "id", "username", "text", "text_length", "party"])               # Reorder columns
    .reset_index(drop=True)                                                                          # Reset index
    .query('text_length > 0')                                                                        # Remove rows with empty tweets
)

print(tweets_df.shape)
tweets_df.head(3)

(19650, 6)


,created_at,id,username,text,text_length,party
0,2022-09-14 19:00:30+00:00,1570125354550845441,SenatorShaheen,My bipartisan resolution with honoring Her Roy...,262,D
1,2022-09-14 19:00:02+00:00,1570125237991002112,SenatorSinema,Were urging Senate leaders of both parties to ...,177,D
2,2022-09-14 19:00:02+00:00,1570125236435034112,SenRickScott,I have major concerns about big tech working w...,235,R


In [8]:
# Get number of null values per column
(tweets_df.isnull().sum().head())

created_at     0
id             0
username       0
text           0
text_length    0
dtype: int64

In [9]:
# Get some info about a df
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19650 entries, 0 to 19649
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   created_at   19650 non-null  datetime64[ns, UTC]
 1   id           19650 non-null  object             
 2   username     19650 non-null  object             
 3   text         19650 non-null  object             
 4   text_length  19650 non-null  int64              
 5   party        19650 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 1.0+ MB


In [10]:
# Describe all numeric columns of dataframe
tweets_df.describe(include="number").T

,count,mean,std,min,25%,50%,75%,max
text_length,19650.0,208.189924,70.246762,4.0,140.0,220.0,273.0,338.0


In [11]:
# Describe all numeric columns of dataframe
tweets_df.describe(include="object").T

,count,unique,top,freq
id,19650,19650,1570125354550845441,1
username,19650,100,SenKevinCramer,200
text,19650,19650,My bipartisan resolution with honoring Her Roy...,1
party,19650,3,R,9753


In [12]:
tweets_df.party.value_counts()

R    9753
D    9497
I     400
Name: party, dtype: int64

In [13]:
# Aggregate tweets by usernames
username_agg_df = (
    tweets_df
    .groupby(["username"])                 # Group by username
    .agg({"text": "count",                 # Count tweets per username
          "text_length": "mean",           # Average tweet length per username
          "created_at": ["min", "max"]})   # Min and max date per username
)

username_agg_df.head(3)

text text_length                created_at  \
               count        mean                       min   
username                                                     
ChrisCoons       199   226.81407 2022-07-08 21:07:45+00:00   
ChrisMurphyCT    193   163.19171 2022-08-10 22:12:00+00:00   
ChrisVanHollen   200   271.17500 2022-07-11 20:31:16+00:00   

                                          
                                     max  
username                                  
ChrisCoons     2022-09-14 18:32:47+00:00  
ChrisMurphyCT  2022-09-13 22:58:07+00:00  
ChrisVanHollen 2022-09-13 20:55:35+00:00

In [14]:
# Create a custom function flatten a df with a multi-index
def flatten_cols(df):
    df.columns = ['_'.join(x) for x in df.columns.to_flat_index()]
    return df.reset_index()

# Aggregate tweets by usernames
username_agg_df = (
    tweets_df
    .groupby(["username"])                 # Group by username
    .agg({"text": "count",                 # Count tweets per username
          "text_length": "mean",           # Average tweet length per username
          "created_at": ["min", "max"]})   # Min and max date per username
    .pipe(flatten_cols)                    # Apply flatten_cols() to entire df 
)

username_agg_df.head(3)


,username,text_count,text_length_mean,created_at_min,created_at_max
0,ChrisCoons,199,226.81407,2022-07-08 21:07:45+00:00,2022-09-14 18:32:47+00:00
1,ChrisMurphyCT,193,163.19171,2022-08-10 22:12:00+00:00,2022-09-13 22:58:07+00:00
2,ChrisVanHollen,200,271.17500,2022-07-11 20:31:16+00:00,2022-09-13 20:55:35+00:00


In [ ]:
# Look at the 5 shortest tweets
tweets_df.sort_values("text_length", ascending=False).tail()

In [ ]:
# tweets_df[tweets_df["text_length"] > 140]
# tweets_df.loc[tweets_df["text_length"] > 140]
# tweets_df.query("text_length > 140")


In [ ]:

# Subset to tweets with length > 140
tweets_df.where(tweets_df["text_length"] > 140)